### PocVid Final Exam

In [2]:
# Import Libraries

import cv2
import time, numpy as np
from skimage import color
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# from skimage import draw
from IPython import display
from skimage import morphology
# from IPython.display import Video
from skimage.color import rgb2gray
import skimage
from skimage.measure import label
# from skimage.measure import regionprops

plt.rc('font', **{'family' : 'DejaVu Sans', 'weight' : 'normal'})
plt.rcParams['font.size'] = 18

In [3]:
# Add text overlay into video frame

def add_text_to_frame(frame, text, position=(30, 30), font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.3, color=(0, 255, 0), thickness=1):
    frame_with_text = frame.copy()
    cv2.putText(frame_with_text, text, position, font, font_scale, color, thickness)
    return frame_with_text

In [4]:
cap = cv2.VideoCapture('hra.mov')

In [5]:
# Get the frame width of the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# Get the frame height of the video
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get the total number of frames in the video
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Get the frames per second (fps) of the video
fps = cap.get(cv2.CAP_PROP_FPS)

# Print the obtained video properties
print("Frame width:", frame_width)
print("Frame height:", frame_height)
print("Length (frames count):", length)
print("Frames per second:", fps)

Frame width: 1142
Frame height: 1260
Length (frames count): 4942
Frames per second: 59.66197183098591


In [6]:
def get_num_of_points(img):
    _, image = cap.read(img)
    image_lab = color.rgb2lab(image)
    points = ((image_lab[:, :, 1] > 8 - 5) * (image_lab[:, :, 1] < 8 + 5)
              * (image_lab[:, :, 2] > -16 - 5) * (image_lab[:, :, 2] < -16 + 5))

    diff_thresholded_processed =  morphology.dilation(morphology.erosion(morphology.closing(morphology.closing(morphology.closing(morphology.remove_small_holes(points, area_threshold=1000), morphology.square(5)))), morphology.square(5)), morphology.square(5))

    label_img = label(diff_thresholded_processed, connectivity=2)

    return len(np.unique(label_img))

In [13]:
cap = cv2.VideoCapture('hra.mov')
MAX_POINTS = get_num_of_points(cap.read()[1])

cv2.namedWindow('Pac-Man', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Pac-Man', 800, 800)  # Adjust the dimensions as needed

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    points = get_num_of_points(frame)

    frame = add_text_to_frame(frame, f'Score: {points}', position=(10,10), color=(0,0,255))

    # Display the frame with bounding boxes
    cv2.imshow('Pac-Man', frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
    time.sleep(.05)

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()